# Dynamic Prompt
<img src="./assets/LC_DynamicPrompts.png" width="500">

## Setup

Load and/or check for needed environmental variables

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env(".env")

GOOGLE_API_KEY=****re4I
LANGSMITH_TRACING=true
LANGSMITH_API_KEY=****ad5c
LANGSMITH_PROJECT=****ials


In [2]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [3]:
from dataclasses import dataclass


@dataclass
class RuntimeContext:
    is_employee: bool
    db: SQLDatabase

In [4]:
from langchain_core.tools import tool
from langgraph.runtime import get_runtime

@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return results."""
    runtime = get_runtime(RuntimeContext)
    db = runtime.context.db

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [5]:
SYSTEM_PROMPT_TEMPLATE = """You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
{table_limits}
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
"""

## Build a Dynamic Prompt
Utilize runtime context and middleware to generate a dynamic prompt.

In [6]:
from langchain.agents.middleware.types import ModelRequest, dynamic_prompt


@dynamic_prompt
def dynamic_system_prompt(request: ModelRequest) -> str:
    if not request.runtime.context.is_employee:
        table_limits = "- Limit access to these tables: Album, Artist, Genre, Playlist, PlaylistTrack, Track."
    else:
        table_limits = ""

    return SYSTEM_PROMPT_TEMPLATE.format(table_limits=table_limits)

Include middleware in `create_agent`.

In [7]:
from langchain.agents import create_agent

agent = create_agent(
    model="google_genai:gemini-2.5-flash",
    tools=[execute_sql],
    middleware=[dynamic_system_prompt],
    context_schema=RuntimeContext,
)

In [8]:
question = "What is the most costly purchase by Frank Harris?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context=RuntimeContext(is_employee=False, db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is the most costly purchase by Frank Harris?
================================== Ai Message ==================================

[{'type': 'text', 'text': 'I am sorry, but I cannot fulfill this request. The available tables (Album, Artist, Genre, Playlist, PlaylistTrack, Track) do not contain information about customers or purchases, so I cannot determine the most costly purchase by Frank Harris.', 'extras': {'signature': 'Co8DAXLI2nxNmgjr1J/X4PQies0ZyXgvj1BhYXrzWuUzPul07lzrUcEZiTTPgVYwMHQorVgmQ4iIXdeQjiKTDvThdTdzUYjDrOdlt3JcOAxIk/3DvKANwEmoMQCOIO5mi/zYiL4IVEpq57nIEvZe427+Sw6ZyFzPLCoHlaeEIdMjjdMoV+y4rokMlyx6H2/5KHIleAg4rSz4N/M7a4tLItTEN8dtmGvgr+F9mojfxa+zZNc2VCcXtJOdWIryP8Dnx7O90Uu9bnG7iTFt8N1CN7VWF/4xTphKhSxdEld9q0v/Aq93DHgc1TvXXplBEP8PnNDkes0IoPGrZUAHb35CMn1tc7Bv/3wJQtRJwvTTLXuyJc1817TpzyYqJpLPAYdhBfVZUMH5Ahi3BGRCAhZIg7r13odcrQhwMwJ0y0b++vwV+EjTu0SCptyDe8TOWmcZCTCnN7qR4mmDxERO4g53Pc9zmUZXdVRUpmVPew4c

In [9]:
question = "What is the most costly purchase by Frank Harris?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context=RuntimeContext(is_employee=True, db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is the most costly purchase by Frank Harris?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (978354d7-d6bd-487f-9731-8239ea938fb4)
 Call ID: 978354d7-d6bd-487f-9731-8239ea938fb4
  Args:
    query: SELECT name FROM sqlite_master WHERE type='table';
================================= Tool Message =================================
Name: execute_sql

[('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',), ('Invoice',), ('InvoiceLine',), ('MediaType',), ('Playlist',), ('PlaylistTrack',), ('Track',)]
================================== Ai Message ==================================
Tool Calls:
  execute_sql (a12f2f6a-fd59-46a3-aaa0-49827401f9ce)
 Call ID: a12f2f6a-fd59-46a3-aaa0-49827401f9ce
  Args:
    query: PRAGMA table_info(Customer);
================================= Tool Message =================================
Name: execute_sql

[